In [ ]:
import pandas as pd
import time
import re
import sys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

# --- FILE CONFIG ---
input_file = "data_usaha_kalipuro.xlsx" 
output_file = "hasil_koordinat_data_usaha_kec-kalipuro_144.xlsx"


# --- KOORDINAT RANGE BANYUWANGI ---
LAT_MIN, LAT_MAX = -8.8, -7.8 
LONG_MIN, LONG_MAX = 113.8, 114.6 

# --- function clean name ---
def clean_name(raw_name):
    if pd.isna(raw_name): return ""
    clean = re.sub(r'<.*?>', '', str(raw_name))
    return " ".join(clean.split())

# --- function get valid coords ---
def get_valid_coords(url):
    match = re.search(r"@([-?\d\.]+),([-?\d\.]+)", url)
    if match:
        lat = float(match.group(1))
        lon = float(match.group(2))
        
        # validation: is it in Banyuwangi?
        if (LAT_MIN <= lat <= LAT_MAX) and (LONG_MIN <= lon <= LONG_MAX):
            # valid coords
            return f"'{lat}", f"'{lon}"
        else:
            print(f"   -> Koordinat diluar Banyuwangi! ({lat}, {lon})")
            return "Cek Manual", "Cek Manual"
            
    return "Cek Manual", "Cek Manual"

# --- SETUP CHROME ---
chrome_options = Options()
# chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# --- LOAD DATA ---
try:
    df = pd.read_excel(input_file)
except FileNotFoundError:
    print("File Excel ga ketemu bro! Cek namanya lagi.")
    sys.exit()

list_latitude = []
list_longitude = []

print("=== Start Scraping ===")

for index, row in df.iterrows():
    # 1. Bersihkan Nama Usaha
    nama_raw = row["nama_usaha"]
    nama_bersih = clean_name(nama_raw)
    
    # 2. Ambil Alamat (Kalau kosong, ganti string kosong)
    alamat_raw = row["alamat"] if pd.notna(row["alamat"]) else ""
    
    # 3. Rakit Keyword Pencarian
    # Format: Nama Bersih + Alamat + Kalipuro + Banyuwangi
    keyword = f"{nama_bersih} {alamat_raw} Kalipuro Banyuwangi"
    
    print(f"[{index+1}] Mencari: {keyword}")

    try:
        # Search di Google Maps
        search_url = f"https://www.google.com/maps/search/{keyword.replace(' ', '+')}"
        driver.get(search_url)
        
        # Tunggu sampai URL berubah ada '@' (Maks 8 detik biar ga kelamaan nunggu)
        WebDriverWait(driver, 8).until(lambda d: "@" in d.current_url)
        time.sleep(1.5) # Jeda dikit biar stabil
        
        # Ambil & Validasi
        lat, lon = get_valid_coords(driver.current_url)
        
        if lat != "Cek Manual":
            print(f"   -> OK: {lat}, {lon}")
        else:
            print("   -> Gagal/Nyasar (Tandai Manual)")

    except Exception:
        # Kalau timeout atau error koneksi
        print("   -> Timeout/Error Search")
        lat, lon = "Cek Manual", "Cek Manual"

    list_latitude.append(lat)
    list_longitude.append(lon)

# --- UPDATE DATAFRAME & SAVE ---
df["Latitude_Hasil"] = list_latitude
df["Longitude_Hasil"] = list_longitude

df.to_excel(output_file, index=False)
driver.quit()

print(f"\nSelesai Bro! Cek file: {output_file}")

Scraping mulai bekerja...
Mencari: Tomoro Coffee and Pastry Sudirman, Banyuwangi, Jl. Panglima Besar Sudirman No.69, Panderejo, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68432
Data Koordinat Ditemukan: -8.2127877, 114.368863
Mencari: Zazito Co-Working Cafe, Jl. Karang Asem Jl. Tunggul Ametung Gg. I No.02, Taman Baru, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68416
Data Koordinat Ditemukan: -8.2242571, 114.3581515
Mencari: D&A Cafe, Jl. Kepiting No.60, Tukangkayu, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68416
Data Koordinat Ditemukan: -8.229541, 114.3679523
Mencari: Sowan coffee, Kp. Melayu, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68419
Data Koordinat Ditemukan: -8.2066671, 114.378224
Mencari: House Of Coffee Banyuwangi, Jl. Brigjen Katamso No.24, Tukangkayu, Kec. Banyuwangi, Kabupaten Banyuwangi, Jawa Timur 68416
Data Koordinat Ditemukan: -8.2271215, 114.3686691
Mencari: Ndalem Coffee And Roastery Kalasan, Jl. Kalasan No.5, Penganjuran, Kec. Banyuwan